In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn_extensions.extreme_learning_machines.elm import GenELMClassifier
from sklearn_extensions.extreme_learning_machines.elm import ELMClassifier

from sklearn_extensions.extreme_learning_machines.random_layer import RBFRandomLayer, MLPRandomLayer
from matplotlib import pyplot
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

C:\Users\91984\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass neg_label=-1, pos_label=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [170]:
def make_classifiers():
    """

    :return:
    """

    names = ["ELM(tanh)", "ELM(sinsq)", "ELM(tribas)", "ELM(hardlim)", "ELM(rbf(0.1)"]

    nh = 500

    # pass user defined transfer func
    sinsq = (lambda x: np.power(np.sin(x), 2.0))
    srhl_sinsq = MLPRandomLayer(n_hidden=nh, activation_func=sinsq)

    # use internal transfer funcs
    srhl_tanh = MLPRandomLayer(n_hidden=nh, activation_func='tanh')
    srhl_tribas = MLPRandomLayer(n_hidden=nh, activation_func='tribas')
    srhl_hardlim = MLPRandomLayer(n_hidden=nh, activation_func='hardlim')

    # use gaussian RBF
    srhl_rbf = RBFRandomLayer(n_hidden=nh*2, rbf_width=0.1, random_state=0)
    log_reg = LogisticRegression()

    classifiers = [GenELMClassifier(hidden_layer=srhl_tanh),
                   GenELMClassifier(hidden_layer=srhl_sinsq),
                   GenELMClassifier(hidden_layer=srhl_tribas),
                   GenELMClassifier(hidden_layer=srhl_hardlim),
                   GenELMClassifier(hidden_layer=srhl_rbf),]

    return names, classifiers

In [171]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

In [172]:
df= pd.read_csv(r'C:\Users\91984\Downloads\Crop_recommendation.csv')
from sklearn import preprocessing    
# Import label encoder
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'label'.
df['label']= label_encoder.fit_transform(df['label'])
target = df['label']
print(df)

        N   P   K  temperature   humidity        ph    rainfall  label
0      90  42  43    20.879744  82.002744  6.502985  202.935536     20
1      85  58  41    21.770462  80.319644  7.038096  226.655537     20
2      60  55  44    23.004459  82.320763  7.840207  263.964248     20
3      74  35  40    26.491096  80.158363  6.980401  242.864034     20
4      78  42  42    20.130175  81.604873  7.628473  262.717340     20
...   ...  ..  ..          ...        ...       ...         ...    ...
2195  107  34  32    26.774637  66.413269  6.780064  177.774507      5
2196   99  15  27    27.417112  56.636362  6.086922  127.924610      5
2197  118  33  30    24.131797  67.225123  6.362608  173.322839      5
2198  117  32  34    26.272418  52.127394  6.758793  127.175293      5
2199  104  18  30    23.603016  60.396475  6.779833  140.937041      5

[2200 rows x 8 columns]


In [173]:
X=df[['N', 'P','K','temperature', 'humidity', 'ph', 'rainfall']]
y=target

from sklearn.preprocessing import PowerTransformer

X = PowerTransformer().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=100)
#X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

In [174]:
y_test = y_test.values.reshape(-1,1)
#y_val  = y_val.values.reshape(-1,1)
y_train = y_train.values.reshape(-1,1 )

In [175]:
names, classifiers = make_classifiers()

In [176]:
for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        score = clf.score(X_train, y_train)
        #score =cross_val_score(clf, X_train, y_train, cv=10)
        print('classifier results')
        print('Model %s score: %s' % (name, score))
        y_pred1 =clf.predict(X_train)      

classifier results
Model ELM(tanh) score: 0.9971590909090909
classifier results
Model ELM(sinsq) score: 0.9960227272727272
classifier results
Model ELM(tribas) score: 0.9977272727272727
classifier results
Model ELM(hardlim) score: 0.9954545454545455
classifier results
Model ELM(rbf(0.1) score: 0.9965909090909091


In [177]:
     # iterate over classifiers
     for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        score = clf.score(X_train, y_train)
        #score =cross_val_score(clf, X_train, y_train, cv=10)
        print('classifier results')
        print('Model %s score: %s' % (name, score))
        y_pred1 =clf.predict(X_train)
        
         ##print ('average Accuracy')
        #print(score.mean())
        #y_pred1 =cross_val_predict(clf,X_train, y_train, cv=10)
        score1=accuracy_score(y_train, y_pred1)
        print (score1)
        cm1 = confusion_matrix(y_train, y_pred1)
        print(cm1)


        
        #print(classification_report(y_train, y_pred1))
        #y_pred2 =cross_val_predict(clf,X_val, y_val, cv=10)
        #y_pred2 =clf.predict(X_val)

        #score2=accuracy_score(y_val, y_pred2)
        #print (score2)
        #cm2 = confusion_matrix(y_val, y_pred2)
        #print(cm2)
       # print(classification_report(y_val, y_pred2))
        y_pred3 =clf.predict(X_test)
       # y_pred3 =cross_val_predict(clf,X_test, y_test, cv=10)
        score3=accuracy_score(y_test, y_pred3)
        print (score3)
        cm3 = confusion_matrix(y_test, y_pred3)
        print(cm3)
        print(classification_report(y_test, y_pred3))

classifier results
Model ELM(tanh) score: 0.9965909090909091
0.9965909090909091
[[82  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 79  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 78  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 75  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 76  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 77  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 74  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 80  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 77  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 80  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 79  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 80  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 80  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 